# LangChain サンプル 2: Retrieval

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** このノートブックは、SageMaker Studioの **Data Science 3.0** カーネルで動作します

### PDF ファイルをロードする DocumentLoader やベクターデータベースなどのインストール

In [5]:
!pip install pymupdf
!pip install numpy==1.26.4 spacy
!python3 -m spacy download ja_core_news_sm
!pip install -qU "langchain-chroma>=0.1.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 38.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: ja-core-news-sm
    Found existing installation: ja_core_news_sm 3.8.0
    Uninstalling ja_core_news_sm-3.8.0:
      Successfully uninstalled ja_core_news_sm-3.8.0
✔ Download and installation successful
You can now load the package via spacy.load('ja_core_news_sm')


### DocumentLoader と　Embed モデルによるベクターデータベース作成のサンプル

In [6]:
from langchain_community.document_loaders import PyMuPDFLoader 
from langchain_aws import BedrockEmbeddings 
from langchain.text_splitter import SpacyTextSplitter
from langchain_community.vectorstores import Chroma 

loader = PyMuPDFLoader("./AnyCompany.pdf") # PDF ファイルを読み込み
documents = loader.load()

text_splitter = SpacyTextSplitter(  # ドキュメント分割用に SpacyTextSplitterを初期化
    chunk_size=300, 
    pipeline="ja_core_news_sm"
)
splitted_documents = text_splitter.split_documents(documents)

embeddings = BedrockEmbeddings( # BedrockEmbeddings を初期化
    model_id = "amazon.titan-embed-text-v1" 
)

database = Chroma(  # Chromaを初期化
    persist_directory="./.data",  # データの保存先を指定
    embedding_function=embeddings  # Embed モデルを指定
)

database.add_documents(  # ドキュメントをデータベースに追加
    splitted_documents,  # 追加するドキュメント
)

print("データベースの作成が完了しました。") 

Created a chunk of size 382, which is longer than the specified 300


データベースの作成が完了しました。


### ベクターデータベースから類似性の高いドキュメントを取得するサンプル

In [ ]:
from langchain_aws import BedrockEmbeddings 
from langchain_chroma import Chroma


embeddings = BedrockEmbeddings(
    model_id = "amazon.titan-embed-text-v1"  
)

database = Chroma(
    persist_directory="./.data", 
    embedding_function=embeddings
)

documents = database.similarity_search("社員が結婚したときにの休暇は何日？") # データベースから類似度の高いドキュメントを取得
print(f"ドキュメントの数: {len(documents)}") # ドキュメントの数を表示

for document in documents:
    print(f"ドキュメントの内容: {document.page_content}") # ドキュメントの内容を表示

### ベクターデータベースから類似性の高いドキュメントを取得してモデルへ問い合わせを行うサンプル

In [ ]:
from langchain_aws import ChatBedrock                
from langchain_core.prompts.prompt import PromptTemplate  
from langchain_core.messages.human import HumanMessage 
from langchain_aws import BedrockEmbeddings  
from langchain_chroma import Chroma


embeddings = BedrockEmbeddings(
    model_id = "amazon.titan-embed-text-v1"  # Bedrock では model_id にする
)

database = Chroma(
    persist_directory="./.data", 
    embedding_function=embeddings
)

query = "社員が結婚したときにの休暇は何日？"

documents = database.similarity_search(query)

documents_string = "" # ドキュメントの内容を格納する変数を初期化

for document in documents:
    documents_string += f"""
---------------------------
{document.page_content}
""" # ドキュメントの内容を追加

prompt = PromptTemplate( # PromptTemplateを初期化
    template="""文章を元に質問に答えてください。 

文章: 
{document}

質問: {query}
""",
    input_variables=["document","query"] # 入力変数を指定
)

chat = ChatBedrock( #  ChatBedrockを初期化
    model_id = "meta.llama3-8b-instruct-v1:0" 
)

result = chat.invoke([
    HumanMessage(content=prompt.format(document=documents_string, query=query))
])

print(result.content)


### ベクターデータベースから類似性の高いドキュメントを取得して　RetrievalQA を使用してモデルへ問い合わせを行うサンプル

In [ ]:
from langchain.chains import RetrievalQA  
from langchain_aws import ChatBedrock            
from langchain_aws import BedrockEmbeddings 
from langchain_chroma import Chroma


chat = ChatBedrock(model_id = "meta.llama3-8b-instruct-v1:0")  

embeddings = BedrockEmbeddings(
     model_id = "amazon.titan-embed-text-v1"  
)

database = Chroma(
    persist_directory="./.data", 
    embedding_function=embeddings
)

retriever = database.as_retriever() # データベースをRetrieverに変換

qa = RetrievalQA.from_llm(         #  RetrievalQAを初期化
    llm=chat,                      #  Chat modelsを指定
    retriever=retriever,           #  Retrieverを指定
    return_source_documents=True   # レスポンスにソースドキュメントを含めるかを指定
)


result = qa.invoke("社員が結婚したときにの休暇は何日？")

print(result["result"]) # レスポンスを表示

print(result["source_documents"]) # ソースドキュメントを表示

### Retriever 課題 : AnyCompany 社の決算説明資料のベクターデータベースを作成して業績について問い合わせてみよう
1. AnyCompany 社の決算説明資料 AnyCompany_IR.pdf からベクターデータベースを作成します。この時、永続化先のディレクトリに `"./.data2"` と指定して下さい。
1. RetrievalQA を使用して、AnyCompany社の営業利益がいくらかを問い合わせてください。出力は英語でもかまいません。
- **上記は、セルを分けてもかまいません。**